# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparametere Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.6s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.4s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.5s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +1.7s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-06 14:14:11,141	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-06 14:14:11,162	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_9e91a598d6c3c0e5.zip' (1.87MiB) to Ray cluster...
2022-01-06 14:14:11,520	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_9e91a598d6c3c0e5.zip'.


(anyscale +11.7s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +11.7s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=5511) == Status ==
(run pid=5511) Current time: 2022-01-06 14:14:18 (running for 00:00:00.12)
(run pid=5511) Memory usage on this node: 2.7/62.0 GiB
(run pid=5511) Using FIFO scheduling algorithm.
(run pid=5511) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.37 GiB objects
(run pid=5511) Result logdir: /home/ray/ray_results/train_func_model_2022-01-06_14-14-17
(run pid=5511) Number of trials: 4/4 (4 PENDING)
(run pid=5511) 
(run pid=5511) 


(ImplicitFunc pid=5582) 2022-01-06 14:14:21,017	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=568, ip=172.31.78.55) 2022-01-06 14:14:21,140	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=562, ip=172.31.69.149) 2022-01-06 14:14:21,148	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=537, ip=172.31.95.94) 2022-01-06 14:14:21,351	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=5582) 2022-01-06 14:14:23,137	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=5647) [14:14:23] task [xgboost.ray]:140299508281888 got new rank 2
(_RemoteRayXGBoostActor pid=5646) [14:14:23] task [xgboost.ray]:140566871572976 got new 

(run pid=5511) == Status ==
(run pid=5511) Current time: 2022-01-06 14:14:24 (running for 00:00:05.56)
(run pid=5511) Memory usage on this node: 3.3/62.0 GiB
(run pid=5511) Using FIFO scheduling algorithm.
(run pid=5511) Resources requested: 20.0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.37 GiB objects
(run pid=5511) Current best trial: fd748_00000 with train-error=0.080844 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0032092244261342474, 'subsample': 0.5456953402106876, 'max_depth': 1, 'nthread': 1, 'n_jobs': 1}
(run pid=5511) Result logdir: /home/ray/ray_results/train_func_model_2022-01-06_14-14-17
(run pid=5511) Number of trials: 4/4 (4 RUNNING)
(run pid=5511) 
(run pid=5511) 
(run pid=5511) 2022-01-06 14:14:24,309	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=5511) 2022-01-06 14:14:24,309	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the c

(ImplicitFunc pid=5582) 2022-01-06 14:14:24,228	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.25 seconds (1.08 pure XGBoost training time).


(run pid=5511) Authenticating
(run pid=5511) Loaded Anyscale authentication token from variable.
(run pid=5511) 
(run pid=5511) 2022-01-06 14:14:25,604	INFO command_runner.py:357 -- Fetched IP: 172.31.78.55
(run pid=5511) 2022-01-06 14:14:25,604	INFO log_timer.py:25 -- NodeUpdater: ins_tg89XjdiD9djfkwcXk8MdP1v: Got IP  [LogTimer=41ms]
(run pid=5511) 2022-01-06 14:14:27,571	INFO command_runner.py:357 -- Fetched IP: 172.31.69.149
(run pid=5511) 2022-01-06 14:14:27,571	INFO log_timer.py:25 -- NodeUpdater: ins_w29yhts53cKc4VWd425F8R1t: Got IP  [LogTimer=103ms]
(run pid=5511) 2022-01-06 14:14:29,291	INFO command_runner.py:357 -- Fetched IP: 172.31.95.94
(run pid=5511) 2022-01-06 14:14:29,291	INFO log_timer.py:25 -- NodeUpdater: ins_6R2GbSNa4kQan5Vp9U8Dw42g: Got IP  [LogTimer=95ms]
(run pid=5511) == Status ==
(run pid=5511) Current time: 2022-01-06 14:14:30 (running for 00:00:11.54)
(run pid=5511) Memory usage on this node: 2.7/62.0 GiB
(run pid=5511) Using FIFO scheduling algorithm.
(run pi

(ImplicitFunc pid=537, ip=172.31.95.94) 2022-01-06 14:14:30,232	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 8.92 seconds (6.45 pure XGBoost training time).
(ImplicitFunc pid=568, ip=172.31.78.55) 2022-01-06 14:14:30,219	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.12 seconds (6.96 pure XGBoost training time).
(ImplicitFunc pid=562, ip=172.31.69.149) 2022-01-06 14:14:30,223	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.11 seconds (6.95 pure XGBoost training time).
(run pid=5511) 2022-01-06 14:14:30,341	INFO tune.py:626 -- Total run time: 12.60 seconds (11.62 seconds for the tuning loop).


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00016056242418745548, 'subsample': 0.9316039415749886, 'max_depth': 7}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
fd748_00000,0.671303,0.057996,0.005046,True,None,None,10,79123a930dbf484a946b34b0721407c5,2022-01-06_14-14-24,1641507264,...,10,"0_eta=0.0032092,max_depth=1,subsample=0.5457",approx,binary:logistic,"[logloss, error]",0.003209,0.545695,1,1,1
fd748_00001,0.691727,0.008787,0.005658,True,None,None,10,64a534de15de4149a4d0d45ac373df8d,2022-01-06_14-14-30,1641507270,...,10,"1_eta=0.00016056,max_depth=7,subsample=0.9316",approx,binary:logistic,"[logloss, error]",0.000161,0.931604,7,1,1
fd748_00002,0.461959,0.035149,0.006099,True,None,None,10,9533d2ef95974537a6929027c6e82fcc,2022-01-06_14-14-30,1641507270,...,10,"2_eta=0.038053,max_depth=2,subsample=0.92326",approx,binary:logistic,"[logloss, error]",0.038053,0.923255,2,1,1
fd748_00003,0.690781,0.015817,0.006858,True,None,None,10,883d29bebcbe4ef88fbb5f4f1d04b21d,2022-01-06_14-14-30,1641507270,...,10,"3_eta=0.0002707,max_depth=7,subsample=0.89321",approx,binary:logistic,"[logloss, error]",0.000271,0.893212,7,1,1


In [9]:
ray.shutdown()

---

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)